In [1]:
# imports:
import time;
import urllib.request
import csv
import os
import itertools
import logging
import json
import numpy as np
import gensim
from bs4 import BeautifulSoup
from urllib.parse import urlencode
from gensim.utils import smart_open, simple_preprocess
from gensim.corpora.wikicorpus import _extract_pages, filter_wiki
from gensim.parsing.preprocessing import STOPWORDS
from xml.etree.cElementTree import iterparse
import matplotlib.pyplot as plt

logging.basicConfig(format='%(levelname)s : %(message)s', level=logging.INFO)
logging.root.level = logging.INFO
model_name="LDA-Cosine"
id2word_wiki=gensim.corpora.Dictionary()

D:\Work\Anaconda\envs\py35\lib\site-packages\gensim\utils.py:855: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
# List Categories:
category_list = ["Mathematics","Technology","Music"]
#category_list = ["Mathematics","Technology","Music","History","Geography","Arts","Health","Nature","Religion","Literature"]
#category_list = ["1977 introductions", "Language" ,"Arts", "Asia", "Asthma", "Automobiles", "BRICS nation", "Belief", "Climate", "Crime", "Culture", "Dance", "Deserts", "Dolls", "Earth", "Engines", "Fishing", "Folklore", "Games", "Geography", "Glass", "Government agencies", "Health", "History", "Humans", "Hygiene", "India", "Industry", "Internet", "Law", "Life", "Literature", "Mathematics", "Matter", "Millionaires", "Music", "Nature", "Nothing", "Parties", "Peace", "Politics", "Religion", "Sexology", "Society", "Songs", "Space", "Technology", "Television", "Transport", "Water sports"]
test_folder =  "./test/"
root_folder = "./Simplex/"
list.sort(category_list)
folder_name=''.join([x[0]+x[1]+x[2] for x in category_list])
root_folder='./'+folder_name+'/'

print("Checking folder : "+folder_name)
if not os.path.exists(root_folder):
    os.mkdir(root_folder)
if not os.path.exists(root_folder+model_name):
    os.mkdir(root_folder+model_name)

    

wiki_bow_path = root_folder+'wiki_bow.mm'
wiki_dict_path = root_folder+'wiki_dict.dict'

Checking folder : MatMusTec


In [3]:
# Download Page Ids:
#https://petscan.wmflabs.org/?language=en&project=wikipedia&depth=1&format=csv&categories=mathematics&doit=Do it!
print ("Scanning CSV")
for cat in category_list:
    if not os.path.exists(root_folder+cat+".csv"):
        url="https://petscan.wmflabs.org/?language=en&project=wikipedia&depth=1&format=csv&doit=Do%20it!&categories="+cat
        urllib.request.urlretrieve(url, root_folder+cat+".csv")
        print("Downloading ",cat+".csv")
    

Scanning CSV


In [4]:
# CSV to XML Download data:

def get_data(ids,output_file):
    url="https://en.wikipedia.org/w/api.php?action=query&prop=revisions&rvprop=content&format=xml&pageids="+ids
    req = urllib.request.urlopen(url)
    if req.getcode() == 200:
        soup = BeautifulSoup(req.read(), 'html.parser')
        s = soup.find_all('page')
        for si in s:
            output_file.write(str(si))

def batch_train(file):
    output_file = open(file.replace(".csv",".xml"), 'a', encoding="utf8")
    output_file.write("<pages>")
                    
    csvReader = csv.reader(open(file,'r'))
    totalRecords = sum(1 for row in csv.reader(open(file,'r',encoding="UTF-8")) )
    print (file.replace(".csv",".xml"),totalRecords)
    start = 0
    end = start + 50
   
    while (start <= totalRecords):
        pageIds = ""
        for row in itertools.islice(csv.reader(open(file,'r',encoding="UTF-8")),start,end):
            pageIds = pageIds + row[2] + "|"
        
        get_data(pageIds,output_file)
        start = end + 1
        end = start + 50
        if end> totalRecords:
            end=totalRecords
            
    output_file.write("</pages>")
    

for path, subdirs, files in os.walk(root_folder):
    for file in files:
        if file.endswith('.csv'):
            if not os.path.exists(root_folder+file.replace(".csv",".xml")):
                print("Downloading ",root_folder+file.replace(".csv",".xml"))
                batch_train(path + file)

print ("Done downloading")



Done downloading


In [5]:
# Train Model
def head(stream, n=10):
    """Convenience fnc: return the first `n` elements of the stream, as plain list."""
    return list(itertools.islice(stream, n))

def my_extract_pages(f):
    elems = (elem for _, elem in iterparse(f, events=("end",)))
    page_tag = "rev"
    for elem in elems:
        if elem.tag == page_tag and elem.text != None:
            text = elem.text
            yield text
            elem.clear()

def tokenize(text):
    return [token for token in simple_preprocess(text) if token not in STOPWORDS]

def iter_wiki(dump_file):
    ignore_namespaces = 'Wikipedia Category File Portal Template MediaWiki User Help Book Draft'.split()
    for text in my_extract_pages(smart_open(dump_file)):
        text = filter_wiki(text)
        tokens = tokenize(text)
        if len(tokens) < 50:
            continue  # ignore short articles and various meta-articles
        yield tokens
        

In [6]:
print("Training Model")
class WikiCorpus(object):
    def __init__(self, dump_file, dictionary, clip_docs=None):
        """
        Parse the first `clip_docs` Wikipedia documents from file `dump_file`.
        Yield each document in turn, as a list of tokens (unicode strings).
        
        """
        self.dump_file = dump_file
        self.dictionary = dictionary
        self.clip_docs = clip_docs
    
    def __iter__(self):
        for tokens in itertools.islice(iter_wiki(self.dump_file), self.clip_docs):
            yield self.dictionary.doc2bow(tokens)
    
    def __len__(self):
        return self.clip_docs


if not (os.path.exists(wiki_bow_path) and  os.path.exists(wiki_dict_path)):
    for path, subdirs, files in os.walk(root_folder):
        del subdirs[:]
        for file in files:
            if file.endswith('.xml'):
                doc_path = path + file
                stream = iter_wiki(doc_path)
                doc_stream = (tokens for tokens in iter_wiki(doc_path))
                id2word_wiki.merge_with(gensim.corpora.Dictionary(doc_stream))

    id2word_wiki.filter_extremes(no_below=10, no_above=0.1)
    
    # create a stream of bag-of-words vectors
    wiki_corpus = WikiCorpus(doc_path, id2word_wiki)
    
    id2word_wiki.save(wiki_dict_path) 
    gensim.corpora.MmCorpus.serialize(wiki_bow_path, wiki_corpus)

id2word_wiki = gensim.corpora.Dictionary.load(wiki_dict_path)
mm_corpus = gensim.corpora.MmCorpus(wiki_bow_path)
clipped_corpus = gensim.utils.ClippedCorpus(mm_corpus, 4000) 
lda_model = gensim.models.LdaModel(clipped_corpus, num_topics=len(category_list), id2word=id2word_wiki, passes=10, alpha='asymmetric')
print("done")

INFO : loading Dictionary object from ./ArtGeoHeaHisLitMatMusNatRelTec/wiki_dict.dict
INFO : loaded ./ArtGeoHeaHisLitMatMusNatRelTec/wiki_dict.dict
INFO : loaded corpus index from ./ArtGeoHeaHisLitMatMusNatRelTec/wiki_bow.mm.index
INFO : initializing corpus reader from ./ArtGeoHeaHisLitMatMusNatRelTec/wiki_bow.mm
INFO : accepted corpus with 1968 documents, 37814 features, 419342 non-zero entries
INFO : using asymmetric alpha [0.20349776650601445, 0.15460680268266819, 0.12465746387131386, 0.10442834231175278, 0.089848016434062955, 0.078840302634700543, 0.070235422324486096, 0.063324036444942805, 0.05765099744942407, 0.052910849340634232]
INFO : using symmetric eta at 2.6445231924683978e-05
INFO : using serial LDA version on this node


Training Model


INFO : running online LDA training, 10 topics, 10 passes over the supplied corpus of 1968 documents, updating model once every 1968 documents, evaluating perplexity every 1968 documents, iterating 50x with a convergence threshold of 0.001000
INFO : -11.625 per-word bound, 3157.8 perplexity estimate based on a held-out corpus of 1968 documents with 804398 words
INFO : PROGRESS: pass 0, at document #1968/1968
INFO : topic #9 (0.053): 0.005*"radar" + 0.003*"energy" + 0.002*"nuclear" + 0.002*"observatory" + 0.002*"electric" + 0.002*"sensor" + 0.002*"gas" + 0.002*"medium" + 0.002*"communication" + 0.002*"machine"
INFO : topic #8 (0.058): 0.003*"software" + 0.003*"video" + 0.003*"engineering" + 0.003*"radio" + 0.003*"internet" + 0.003*"mobile" + 0.003*"content" + 0.002*"users" + 0.002*"user" + 0.002*"air"
INFO : topic #2 (0.125): 0.005*"mobile" + 0.004*"engineering" + 0.004*"digital" + 0.003*"software" + 0.003*"user" + 0.002*"video" + 0.002*"technical" + 0.002*"devices" + 0.002*"observatory"

Wall time: 4min 12s
done


In [17]:
def calculate_centroid(topic_docs):
    test_doc = [tokens for tokens in iter_wiki(topic_docs)]
    part = [lda_model[id2word_wiki.doc2bow(tokens)] for tokens in test_doc]
    
    topic_dic={}
    
    for i in range(len(category_list)):
        topic_dic[i]=0
        
    for doc in part:
        for p in doc:
            topic_dic[p[0]] += p[1]
    
    centroid = [(x, topic_dic[x]/len(part)) for x in range(len(category_list))]
    return centroid

print("Calcuating centroid")

centroids_dict={}
for path, subdirs, files in os.walk(root_folder):
    del subdirs[:]
    for file in files:
        if file.endswith('.xml'):
            doc_path = path + file
            centroid = calculate_centroid(doc_path)
            centroids_dict[file.replace(".xml","")]=centroid

print("Centroid calculation done")

Calcuating centroid
./ArtGeoHeaHisLitMatMusNatRelTec/Arts.xml
./ArtGeoHeaHisLitMatMusNatRelTec/Geography.xml
./ArtGeoHeaHisLitMatMusNatRelTec/Health.xml
./ArtGeoHeaHisLitMatMusNatRelTec/History.xml
./ArtGeoHeaHisLitMatMusNatRelTec/Literature.xml
./ArtGeoHeaHisLitMatMusNatRelTec/Mathematics.xml
./ArtGeoHeaHisLitMatMusNatRelTec/Music.xml
./ArtGeoHeaHisLitMatMusNatRelTec/Nature.xml
./ArtGeoHeaHisLitMatMusNatRelTec/Religion.xml
./ArtGeoHeaHisLitMatMusNatRelTec/Technology.xml
Centroid calculation done


In [42]:
def draw_graph(x_label,y,file,text_data,topic):
    x = np.arange(len(x_label))  # the x locations for the groups
    width = 0.1       # the width of the bars

    fig, ax = plt.subplots()
    fig.set_figheight(6)
    fig.set_figwidth(8)
    rects = ax.bar(x, y, width, color='blue')
    ax.set_ylabel('Probabilities')
    ax.set_xlabel('Categories')
    ax.set_title('Topic Distribution of: ' + topic)
    ax.title.set_position([.5, 1.2])
    ax.set_ylim([0,1])
    
    def autolabel(rects):
        """
        Attach a text label above each bar displaying its height
        """
        for rect in rects:
            height = rect.get_height()
            ax.text(rect.get_x() - rect.get_width()*2, 1.05 * height,round(height,2))
    

    autolabel(rects)
    plt.tight_layout(pad=6)
    plt.xticks(x,category_list,rotation=90)
    
    plt.savefig(file.replace(".xml","_")+str(len(category_list))+'.png')
    plt.close('all')

In [43]:
# Test 
def get_part(testFile):
    test_doc = [tokens for tokens in iter_wiki(testFile)]
    part = [lda_model[id2word_wiki.doc2bow(tokens)] for tokens in test_doc]
    return part

results=""
print("Testing results")
for path, subdirs, files in os.walk(test_folder):
    for file in files:
        if file.endswith('.xml'):
            doc_path = test_folder + file
            path=get_part(doc_path)
            results+=file.replace(".xml","")+"\n\n"
            graph_data=[]
            text_data=""
            for topic in category_list:
                
                cos_dis=np.mean([gensim.matutils.cossim(p1, p2) for p1, p2 in zip([centroids_dict[topic]], path)])
                text_data +=topic+":"+str(cos_dis)+"\n"
                
                graph_data.append(np.mean([gensim.matutils.cossim(p1, p2) for p1, p2 in zip([centroids_dict[topic]], path)]))
            results+=text_data+"\n\n"
                
            draw_graph(list(centroids_dict.keys()),graph_data,root_folder+model_name+'/'+file,text_data,file)
output_file = open(root_folder+model_name+"/"+model_name+str(len(category_list))+".txt", 'w', encoding="utf8")
output_file.write(results)
output_file.close()
print(" Done ")

Testing results
 Done 
